In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

https://www.dropbox.com/s/d0ycz48z8qj0dws/mergedissue.csv.zip?dl=0
 
This contains all US issued bonds with ISINs to identify the bonds and CUSIP for borrower identification.
 
The bonds can be merged with TRACE Enhanced (this is already cleaned) for bond prices.
 
For bond data this is all you need! For borrower data, you want to use the merged CRSP/Compustat database. Both Trace Enhanced and CRSP/Compustat are available via WRDS.
 
Now you can start working with the data and prepare the trading data etc. for the modeling.
 
Did you make some progress in the literature on bond pricing to get a better understanding how they model bond returns? You want to predict returns eventually (not prices).

In [2]:
df = pd.read_csv(r'/Users/ishantchaudhary/REPOS/Master Thesis Bond Prices Prediction/data/raw/mergedissue.csv')

/var/folders/4b/l73ksqn90ls5dx9x6n2k93xc0000gn/T/ipykernel_35609/3855846600.py:1: DtypeWarning: Columns (8,27,42,43,48,56,57,63,81,85,97,98,99,100,101,106,107,108,111,112,116,126,127,128,129,130,131,132,133,134,135,136,137,138,140,143,145,146,147,148,161,162,167,189,190,191,192,193,195,196,199,200,201,203,204,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/Users/ishantchaudhary/REPOS/Master Thesis Bond Prices Prediction/data/raw/mergedissue.csv')


In [3]:
df.columns

Index(['issue_id', 'issuer_id', 'prospectus_issuer_name', 'issuer_cusip',
       'issue_cusip', 'issue_name', 'maturity', 'security_level',
       'security_pledge', 'enhancement',
       ...
       'subsidiary_redesignation', 'subsidiary_guarantee',
       'sales_leaseback_sub', 'liens_sub', 'fixed_charge_coverage_sub',
       'leverage_test_sub', 'unit_cusip', 'total_units_offered',
       'principal_amt_per_unit', 'allocated_offering_price_unit'],
      dtype='object', length=224)

In [4]:
df.head()

,issue_id,issuer_id,prospectus_issuer_name,issuer_cusip,issue_cusip,issue_name,maturity,security_level,security_pledge,enhancement,...,subsidiary_redesignation,subsidiary_guarantee,sales_leaseback_sub,liens_sub,fixed_charge_coverage_sub,leverage_test_sub,unit_cusip,total_units_offered,principal_amt_per_unit,allocated_offering_price_unit
0,1.0,3.0,AAR CORP,000361,AA3,NT,2001-11-01,SEN,NaN,N,...,N,N,N,N,N,N,NaN,NaN,NaN,NaN
1,2.0,3.0,AAR CORP,000361,AB1,NT,2003-10-15,SEN,NaN,N,...,N,N,Y,N,N,N,NaN,NaN,NaN,NaN
2,3.0,40263.0,ABN AMRO BK N V N Y BRH,00077D,AB5,MTN,1996-01-12,SEN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,40263.0,ABN AMRO BK N V N Y BRH,00077D,AF6,SUB DEP NT SER B,2009-08-01,SENS,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,40263.0,ABN AMRO BK N V N Y BRH,00077T,AA2,SUB DEP NT SER B,2023-05-15,SUB,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# function to remove columns on basis of conditions

def remove_empty_columns(data):
    # Calculate the total number of rows in the DataFrame
    total_rows = len(data)

    # Iterate over each column in the DataFrame
    for column in data.columns:
        # Count the number of empty values in the column
        num_empty_values = data[column].isnull().sum()

        # Calculate the percentage of empty values in the column
        percentage_empty = (num_empty_values / total_rows) * 100

        # Check if more than 80% of the values are empty or if there is only one unique value
        if percentage_empty > 80 or data[column].nunique() == 1:
            # print(column)
            # Remove the column from the DataFrame
            data.drop(column, axis=1, inplace=True)

    return data


In [6]:
df.shape

(561184, 224)

In [7]:
clean_data = remove_empty_columns(df)
clean_data.shape


(561184, 71)

In [8]:
categories = ['convertible','foreign_currency','putable','private_placement','perpetual','exchangeable','preferred_security','rule_144a']

In [9]:
for cat in categories:
    print(cat)
    print(clean_data[cat].value_counts())
    clean_data = clean_data[clean_data[cat]=='N']
    print(clean_data[cat].value_counts())
    clean_data = clean_data.drop(cat, axis=1)


convertible
N    554156
Y      6103
Name: convertible, dtype: int64
N    554156
Name: convertible, dtype: int64
foreign_currency
N    547550
Y      6606
Name: foreign_currency, dtype: int64
N    547550
Name: foreign_currency, dtype: int64
putable
N    545797
Y      1599
Name: putable, dtype: int64
N    545797
Name: putable, dtype: int64
private_placement
N    545113
Y       684
Name: private_placement, dtype: int64
N    545113
Name: private_placement, dtype: int64
perpetual
N    543222
Y      1891
Name: perpetual, dtype: int64
N    543222
Name: perpetual, dtype: int64
exchangeable
N    489311
Y     53911
Name: exchangeable, dtype: int64
N    489311
Name: exchangeable, dtype: int64
preferred_security
N    462157
Y     27154
Name: preferred_security, dtype: int64
N    462157
Name: preferred_security, dtype: int64
rule_144a
N    435679
Y     26478
Name: rule_144a, dtype: int64
N    435679
Name: rule_144a, dtype: int64


In [10]:
clean_data.shape

(435679, 63)

In [11]:
clean_data.columns

Index(['issue_id', 'issuer_id', 'prospectus_issuer_name', 'issuer_cusip',
       'issue_cusip', 'issue_name', 'maturity', 'security_level',
       'enhancement', 'coupon_type', 'mtn', 'asset_backed', 'yankee',
       'canadian', 'oid', 'slob', 'issue_offered_global', 'settlement_type',
       'gross_spread', 'selling_concession', 'comp_neg_exch_deal',
       'rule_415_reg', 'sec_reg_type1', 'offering_amt', 'offering_date',
       'offering_price', 'offering_yield', 'delivery_date', 'unit_deal',
       'form_of_own', 'denomination', 'principal_amt', 'covenants', 'defeased',
       'defaulted', 'tender_exch_offer', 'redeemable', 'refund_protection',
       'overallotment_opt', 'announced_call', 'active_issue',
       'dep_eligibility', 'bond_type', 'subsequent_data', 'isin', 'fungible',
       'complete_cusip', 'action_type', 'effective_date', 'action_price',
       'action_amount', 'amount_outstanding', 'greater_of', 'lesser_of',
       'see_note', 'dated_date', 'first_interest_date', '

In [12]:
# import ydata_profiling

# profile = ydata_profiling.ProfileReport(clean_data)
# profile.to_file("clean_data_report_2.html")

In [13]:
# Filtering all the bonds that were issues after 2012-02-07

clean_data['offering_date'] = pd.to_datetime(clean_data['offering_date']).dt.date

# Filter rows where 'offering_date' is greater than '2010-01-01'
red_data = clean_data[clean_data['offering_date'] > pd.to_datetime('2012-02-07')]

# Retrieve the shape of ten_yr_data
red_data.shape


/Users/ishantchaudhary/Anaconda/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


(189183, 63)

In [14]:
# Convert offering_date and maturity columns to datetime
red_data['offering_date'] = pd.to_datetime(red_data['offering_date'])
red_data['maturity'] = pd.to_datetime(red_data['maturity'])

# Calculate the difference in years between offering_date and maturity
red_data['years_to_maturity'] = (red_data['maturity'] - red_data['offering_date']).dt.days / 365

/var/folders/4b/l73ksqn90ls5dx9x6n2k93xc0000gn/T/ipykernel_35609/1263813692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_data['offering_date'] = pd.to_datetime(red_data['offering_date'])
/var/folders/4b/l73ksqn90ls5dx9x6n2k93xc0000gn/T/ipykernel_35609/1263813692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_data['maturity'] = pd.to_datetime(red_data['maturity'])
/var/folders/4b/l73ksqn90ls5dx9x6n2k93xc0000gn/T/ipykernel_35609/1263813692.py:6: SettingWithCopyWarning: 
A value is trying t

In [15]:
# Filter out rows with maturity less than 1 year or more than 30 years
red_data = red_data[(red_data['years_to_maturity'] >= 1) & (red_data['years_to_maturity'] <= 30)]
red_data.shape

(180214, 64)

In [16]:
red_data['offering_amt'].isna().value_counts()

False    180208
True          6
Name: offering_amt, dtype: int64

In [17]:
red_data = red_data[red_data['offering_amt'].notna()]
red_data.shape

(180208, 64)

In [18]:
red_data = red_data[red_data['offering_date'].notna()]
red_data.shape

(180208, 64)

In [19]:
red_data = red_data[red_data['coupon'].notna()]
red_data.shape

(139024, 64)

In [20]:
red_data = red_data[red_data['maturity'].notna()]
red_data.shape

(139024, 64)

In [22]:
# Taking only the Corporate Debentures

red_data= red_data[red_data['bond_type']=='CDEB']
red_data.shape

(10454, 64)

In [27]:
# Filtering only the bonds that are issued globaly
red_data= red_data[red_data['issue_offered_global']=='Y']
red_data.shape

(6087, 64)

In [24]:
red_data['offering_yield'].isna().value_counts()

False    5029
True     1058
Name: offering_yield, dtype: int64

In [25]:
red_data['complete_cusip'].nunique()

6087

In [26]:
unique_values = red_data['complete_cusip'].astype(str).unique().tolist()
print(len(unique_values))

6087


In [30]:
with open('cusip_codes_reduced_6k.txt', 'w') as file:
    for value in unique_values:
        file.write(value + '\n')

In [29]:
red_data.shape

(6087, 64)

In [32]:
# Save the DataFrame as a pickle file
# red_data.to_pickle("mergent_final_6k.pkl")

In [28]:
# print(tabulate(red_data.head(50), headers='keys', tablefmt='fancy_grid'))